In [ ]:
from google.colab import driveimport osdef mount_drive():    drive.mount('/content/drive')

In [ ]:
# Clone the repository from Hugging Face!git clone https://huggingface.co/Laxhar/noobai-XL-1.1.git

In [ ]:
# Install required libraries!pip install torch torchvision transformers diffusers accelerate safetensors huggingface_hub

In [ ]:
# Define a function to load the modelfrom diffusers import StableDiffusionPipelinefrom transformers import AutoTokenizerfrom huggingface_hub import hf_hub_downloadimport torchdef load_model():    model_path = "./noobai-XL-1.1"    # Load the text-to-image pipeline    pipeline = StableDiffusionPipeline.from_pretrained(        model_path,        torch_dtype=torch.float16    ).to("cuda")    # Enable gradient checkpointing to reduce memory usage    pipeline.enable_gradient_checkpointing()    # Update tokenizers to remove limits    tokenizer_path = os.path.join(model_path, "tokenizer")    tokenizer_2_path = os.path.join(model_path, "tokenizer_2")    for path in [tokenizer_path, tokenizer_2_path]:        config_file = os.path.join(path, "tokenizer_config.json")        with open(config_file, "r") as f:            config = f.read()        config = config.replace("1024", "4096")  # Increase limit        with open(config_file, "w") as f:            f.write(config)    return pipeline

In [ ]:
# Create UI for interacting with the modelfrom ipywidgets import interact, FloatSlider, IntSlider, Text, Dropdownfrom PIL import Imagedef generate_image(pipeline, prompt, steps, sampler, seed, cfg_scale, width, height, hires_scale, hires_steps):    from diffusers.schedulers import LMSDiscreteScheduler, DDIMScheduler, PNDMScheduler    # Configure sampler    if sampler == "LMS":        pipeline.scheduler = LMSDiscreteScheduler.from_config(pipeline.scheduler.config)    elif sampler == "DDIM":        pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)    elif sampler == "PNDM":        pipeline.scheduler = PNDMScheduler.from_config(pipeline.scheduler.config)    # Set seed    if seed is not None:        torch.manual_seed(seed)    # Generate image    image = pipeline(
        prompt,        num_inference_steps=steps,        guidance_scale=cfg_scale,        width=width,        height=height    ).images[0]    if hires_scale:        from diffusers import ImageUpscaler        upscaler = ImageUpscaler.from_pretrained("8x-NKMD-Superscale")        image = upscaler(image, steps=hires_steps, scale=hires_scale)    return image# Interactive functionpipeline = load_model()def interactive_demo():    interact(
        lambda prompt, steps, sampler, seed, cfg_scale, width, height, hires_scale, hires_steps: generate_image(
            pipeline, prompt, steps, sampler, seed, cfg_scale, width, height, hires_scale, hires_steps
        ),        prompt=Text(description="Prompt"),        steps=IntSlider(value=50, min=10, max=150, step=10, description="Steps"),        sampler=Dropdown(options=["LMS", "DDIM", "PNDM"], description="Sampler"),        seed=IntSlider(value=42, min=0, max=1000, step=1, description="Seed"),        cfg_scale=FloatSlider(value=7.5, min=1.0, max=20.0, step=0.5, description="CFG Scale"),        width=IntSlider(value=512, min=512, max=2048, step=64, description="Width"),        height=IntSlider(value=512, min=512, max=2048, step=64, description="Height"),        hires_scale=FloatSlider(value=2.0, min=1.0, max=4.0, step=0.1, description="Hires Scale"),        hires_steps=IntSlider(value=20, min=10, max=50, step=5, description="Hires Steps"),    )# Start the interactive demointeractive_demo()